In [2]:
import tensorflow as tf
import numpy as np
from  skimage import transform
from matplotlib import pyplot
from tensorflow.keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
from sklearn.metrics import precision_score
import keras.backend as K

In [3]:
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
print(x_train.shape)

(60000, 28, 28)


In [5]:
# one hot encode target values
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [6]:
x_train_reshape_arch1 = x_train.reshape(60000, 784).astype('float32')#use 60k for training
x_test_reshape_arch1 = x_test.reshape(10000, 784).astype('float32')
# normalize to range 0-1
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train_reshape_arch1 = x_train_reshape_arch1 / 255.0
x_test_reshape_arch1 = x_test_reshape_arch1 / 255.0

#Architecture 1

In [7]:
print(x_train.shape)
print(x_train_reshape_arch1.shape)

(60000, 28, 28)
(60000, 784)


In [8]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [20]:
#model 1 based on architectuer 1
#using adam as optimizer and 50 epoches
model1 = Sequential()
model1.add(Dense(128, activation='relu',input_shape=(784,))) #we use only 1 channel
model1.add(Dense(128, activation='relu',))
model1.add(Dense(10, activation='softmax'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m])
history1 = model1.fit(x_train_reshape_arch1, y_train, epochs=50, batch_size=16, validation_data=(x_test_reshape_arch1, y_test), verbose=1)
_, acc_1,F1_1,Precison_1 = model1.evaluate(x_test_reshape_arch1,y_test, verbose=1)
print('model1 has accuracy of :'+str(acc_1*100.0))
print('model1 has f1 score  :' +str(F1_1 * 100.0))
print('model1 has Precison :' +str(Precison_1 * 100.0))

Epoch 1/50
3750/3750 [==============================] - 45s 12ms/step - loss: 0.4812 - accuracy: 0.8259 - f1_m: 0.8189 - precision_m: 0.8684 - val_loss: 0.4450 - val_accuracy: 0.8477 - val_f1_m: 0.8464 - val_precision_m: 0.8775
Epoch 2/50
3750/3750 [==============================] - 11s 3ms/step - loss: 0.3664 - accuracy: 0.8650 - f1_m: 0.8640 - precision_m: 0.8942 - val_loss: 0.3906 - val_accuracy: 0.8631 - val_f1_m: 0.8594 - val_precision_m: 0.8923
Epoch 3/50
3750/3750 [==============================] - 11s 3ms/step - loss: 0.3305 - accuracy: 0.8774 - f1_m: 0.8770 - precision_m: 0.9021 - val_loss: 0.3901 - val_accuracy: 0.8617 - val_f1_m: 0.8628 - val_precision_m: 0.8837
Epoch 4/50
3750/3750 [==============================] - 11s 3ms/step - loss: 0.3071 - accuracy: 0.8870 - f1_m: 0.8869 - precision_m: 0.9085 - val_loss: 0.3573 - val_accuracy: 0.8737 - val_f1_m: 0.8746 - val_precision_m: 0.8989
Epoch 5/50
3750/3750 [==============================] - 11s 3ms/step - loss: 0.2899 - accur

In [19]:
#model 2 based on Arcitecture 1 
#using SGD as optimizer and 50 epoches
model2 = Sequential()
model2.add(Dense(128, activation='relu',input_shape=(784,))) #we use only 1 channel
model2.add(Dense(128, activation='relu',))
model2.add(Dense(10, activation='softmax'))
opt = SGD(lr=0.01, momentum=0.9)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history2 = model2.fit(x_train_reshape_arch1, y_train, epochs=50, batch_size=16, validation_data=(x_test_reshape_arch1, y_test), verbose=1)
_, acc_2,= model2.evaluate(x_test_reshape_arch1,y_test, verbose=1)
print('model2 has accuracy of :'+str(acc_2*100.0))

Epoch 1/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5137 - accuracy: 0.8117 - val_loss: 0.4643 - val_accuracy: 0.8333
Epoch 2/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.3854 - accuracy: 0.8579 - val_loss: 0.4021 - val_accuracy: 0.8559
Epoch 3/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.3487 - accuracy: 0.8703 - val_loss: 0.3835 - val_accuracy: 0.8593
Epoch 4/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.3256 - accuracy: 0.8790 - val_loss: 0.4027 - val_accuracy: 0.8538
Epoch 5/50
3750/3750 [==============================] - 9s 2ms/step - loss: 0.3070 - accuracy: 0.8845 - val_loss: 0.3803 - val_accuracy: 0.8671
Epoch 6/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.2954 - accuracy: 0.8895 - val_loss: 0.3689 - val_accuracy: 0.8679
Epoch 7/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.2841 - accuracy: 0.8946 - val_loss: 0.3971 - val_acc

In [22]:
#model 3 based on architecture 1
#using sigmoid as activation funtion
#using dropout o.1
#using adam as optimizer
model3 = Sequential()
model3.add(Dense(128, activation='sigmoid',input_shape=(784,))) #we use only 1 channel
model3.add(Dense(128, activation='sigmoid'))
model3.add(Dropout(0.1))
model3.add(Dense(10, activation='softmax'))
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history3 = model3.fit(x_train_reshape_arch1, y_train, epochs=50, batch_size=16, validation_data=(x_test_reshape_arch1, y_test), verbose=1)
_, acc_3= model3.evaluate(x_test_reshape_arch1,y_test, verbose=1)
print('model3 has accuracy of :'+str(acc_3*100.0))

Epoch 1/50
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5506 - accuracy: 0.8044 - val_loss: 0.4679 - val_accuracy: 0.8286
Epoch 2/50
3750/3750 [==============================] - 12s 3ms/step - loss: 0.3846 - accuracy: 0.8602 - val_loss: 0.4047 - val_accuracy: 0.8528
Epoch 3/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.3471 - accuracy: 0.8733 - val_loss: 0.3851 - val_accuracy: 0.8601
Epoch 4/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.3225 - accuracy: 0.8818 - val_loss: 0.3579 - val_accuracy: 0.8668
Epoch 5/50
3750/3750 [==============================] - 11s 3ms/step - loss: 0.3046 - accuracy: 0.8875 - val_loss: 0.3595 - val_accuracy: 0.8693
Epoch 6/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.2904 - accuracy: 0.8917 - val_loss: 0.3550 - val_accuracy: 0.8686
Epoch 7/50
3750/3750 [==============================] - 10s 3ms/step - loss: 0.2757 - accuracy: 0.8974 - val_loss: 0.3418 - val_ac

In [17]:
#model 4 based on architecture 1 
#using sigmoid as activation funtion
#using adam as optimizer
#using 100 as batch size
#using 50 as epochs
model4 = Sequential()
model4.add(Dense(128, activation='sigmoid',input_shape=(784,))) #we use only 1 channel
model4.add(Dense(128, activation='sigmoid'))
model4.add(Dropout(0.1))
model4.add(Dense(10, activation='softmax'))
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history3 = model4.fit(x_train_reshape_arch1, y_train, epochs=60, batch_size=100, validation_data=(x_test_reshape_arch1, y_test), verbose=1)
_, acc_4= model4.evaluate(x_test_reshape_arch1,y_test, verbose=1)
print('model4 has accuracy of :'+str(acc_4*100.0))

Epoch 1/60
600/600 [==============================] - 2s 3ms/step - loss: 0.7760 - accuracy: 0.7369 - val_loss: 0.4926 - val_accuracy: 0.8249
Epoch 2/60
600/600 [==============================] - 2s 3ms/step - loss: 0.4388 - accuracy: 0.8425 - val_loss: 0.4372 - val_accuracy: 0.8405
Epoch 3/60
600/600 [==============================] - 2s 3ms/step - loss: 0.3908 - accuracy: 0.8590 - val_loss: 0.4233 - val_accuracy: 0.8456
Epoch 4/60
600/600 [==============================] - 2s 3ms/step - loss: 0.3640 - accuracy: 0.8690 - val_loss: 0.3814 - val_accuracy: 0.8643
Epoch 5/60
600/600 [==============================] - 2s 3ms/step - loss: 0.3457 - accuracy: 0.8745 - val_loss: 0.3726 - val_accuracy: 0.8646
Epoch 6/60
600/600 [==============================] - 2s 3ms/step - loss: 0.3302 - accuracy: 0.8795 - val_loss: 0.3646 - val_accuracy: 0.8689
Epoch 7/60
600/600 [==============================] - 2s 3ms/step - loss: 0.3184 - accuracy: 0.8837 - val_loss: 0.3612 - val_accuracy: 0.8712
Epoch 

#using 2nd Architecture
#developing 4 diffrent modales with diffrent epoches and optimizers like the first selected Architecture

In [14]:
#using 2nd architecture
#using adam as optimizer 

model2_1 = Sequential()
model2_1.add(Conv2D(64,(2,2),activation='relu',padding='same',input_shape=(28,28,1)))
#pooling
model2_1.add(MaxPooling2D((2, 2)))
#dropout
model2_1.add(Dropout(0.5))
model2_1.add(Conv2D(64,(2,2),activation='relu',padding='same'))
#dropout
model2_1.add(Dropout(0.25))
#flat
model2_1.add(Flatten())
model2_1.add(Dense(64, activation='relu'))
model2_1.add(Dropout(0.25))
model2_1.add(Dense(10, activation='softmax'))
model2_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m])
history2_1 = model2_1.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=1)
_, acc_2_1,F1_2_1,Precison_2_1 = model2_1.evaluate(x_test,y_test, verbose=1)
print('model2_1 has accuracy of :'+str(acc_2_1*100.0))
print('model2_1 has f1 score  :' +str(F1_2_1 * 100.0))
print('model2_1 has Precison :' +str(Precison_2_1 * 100.0))

Epoch 1/50
469/469 [==============================] - 10s 9ms/step - loss: 0.5643 - accuracy: 0.7987 - f1_m: 0.7867 - precision_m: 0.8484 - val_loss: 0.3902 - val_accuracy: 0.8602 - val_f1_m: 0.8605 - val_precision_m: 0.8884
Epoch 2/50
469/469 [==============================] - 4s 9ms/step - loss: 0.3822 - accuracy: 0.8648 - f1_m: 0.8629 - precision_m: 0.8954 - val_loss: 0.3151 - val_accuracy: 0.8824 - val_f1_m: 0.8851 - val_precision_m: 0.9072
Epoch 3/50
469/469 [==============================] - 4s 9ms/step - loss: 0.3391 - accuracy: 0.8767 - f1_m: 0.8763 - precision_m: 0.9025 - val_loss: 0.2954 - val_accuracy: 0.8920 - val_f1_m: 0.8938 - val_precision_m: 0.9124
Epoch 4/50
469/469 [==============================] - 4s 9ms/step - loss: 0.3082 - accuracy: 0.8877 - f1_m: 0.8882 - precision_m: 0.9111 - val_loss: 0.2762 - val_accuracy: 0.8992 - val_f1_m: 0.9005 - val_precision_m: 0.9173
Epoch 5/50
469/469 [==============================] - 4s 9ms/step - loss: 0.2860 - accuracy: 0.8942 - f

In [15]:
#using 2nd architecture
#using SGD as optimizer with learning rate 0.01 

model2_2 = Sequential()
model2_2.add(Conv2D(64,(2,2),activation='relu',padding='same',input_shape=(28,28,1)))
#pooling
model2_2.add(MaxPooling2D((2, 2)))
#dropout
model2_2.add(Dropout(0.5))
model2_2.add(Conv2D(64,(2,2),activation='relu',padding='same'))
#dropout
model2_2.add(Dropout(0.25))
#flat
model2_2.add(Flatten())
model2_2.add(Dense(64, activation='relu'))
model2_2.add(Dropout(0.25))
model2_2.add(Dense(10, activation='softmax'))
sgd_optimizer=SGD(lr=0.01, momentum=0.9)
model2_2.compile(optimizer=SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
history2_2 = model2_2.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=1)
_, acc_2_2= model2_2.evaluate(x_test,y_test, verbose=1)
print('model2_2 has accuracy of :'+str(acc_2_2*100.0))

Epoch 1/50
469/469 [==============================] - 4s 9ms/step - loss: 1.1540 - accuracy: 0.5814 - val_loss: 0.7212 - val_accuracy: 0.7398
Epoch 2/50
469/469 [==============================] - 4s 8ms/step - loss: 0.7400 - accuracy: 0.7285 - val_loss: 0.6172 - val_accuracy: 0.7748
Epoch 3/50
469/469 [==============================] - 4s 8ms/step - loss: 0.6589 - accuracy: 0.7584 - val_loss: 0.5562 - val_accuracy: 0.7957
Epoch 4/50
469/469 [==============================] - 4s 8ms/step - loss: 0.6128 - accuracy: 0.7748 - val_loss: 0.5490 - val_accuracy: 0.7986
Epoch 5/50
469/469 [==============================] - 4s 8ms/step - loss: 0.5811 - accuracy: 0.7866 - val_loss: 0.5089 - val_accuracy: 0.8140
Epoch 6/50
469/469 [==============================] - 4s 8ms/step - loss: 0.5608 - accuracy: 0.7956 - val_loss: 0.5106 - val_accuracy: 0.8107
Epoch 7/50
469/469 [==============================] - 4s 8ms/step - loss: 0.5417 - accuracy: 0.8017 - val_loss: 0.4822 - val_accuracy: 0.8236
Epoch 

In [25]:
#using 2nd architecture
#using SGD as optimizer with learning rate 0.001 

model2_3 = Sequential()
model2_3.add(Conv2D(64,(2,2),activation='relu',padding='same',input_shape=(28,28,1)))
#pooling
model2_3.add(MaxPooling2D((2, 2)))
#dropout
model2_3.add(Dropout(0.5))
model2_3.add(Conv2D(64,(2,2),activation='relu',padding='same'))
#dropout
model2_3.add(Dropout(0.25))
#flat
model2_3.add(Flatten())
model2_3.add(Dense(64, activation='relu'))
model2_3.add(Dropout(0.25))
model2_3.add(Dense(10, activation='softmax'))
model2_3.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history2_3 = model2_3.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=1)
_, acc_2_3,= model2_3.evaluate(x_test,y_test, verbose=1)
print('model2_3 has accuracy of :'+str(acc_2_3*100.0))

Epoch 1/50
469/469 [==============================] - 4s 9ms/step - loss: 2.2345 - accuracy: 0.2102 - val_loss: 2.1288 - val_accuracy: 0.4567
Epoch 2/50
469/469 [==============================] - 4s 8ms/step - loss: 1.9201 - accuracy: 0.3912 - val_loss: 1.5778 - val_accuracy: 0.5452
Epoch 3/50
469/469 [==============================] - 4s 8ms/step - loss: 1.4168 - accuracy: 0.5033 - val_loss: 1.1092 - val_accuracy: 0.6257
Epoch 4/50
469/469 [==============================] - 4s 8ms/step - loss: 1.1458 - accuracy: 0.5720 - val_loss: 0.9324 - val_accuracy: 0.6592
Epoch 5/50
469/469 [==============================] - 4s 8ms/step - loss: 1.0247 - accuracy: 0.6182 - val_loss: 0.8492 - val_accuracy: 0.6886
Epoch 6/50
469/469 [==============================] - 4s 8ms/step - loss: 0.9488 - accuracy: 0.6465 - val_loss: 0.7945 - val_accuracy: 0.7065
Epoch 7/50
469/469 [==============================] - 4s 8ms/step - loss: 0.8948 - accuracy: 0.6679 - val_loss: 0.7518 - val_accuracy: 0.7238
Epoch 